# 1 Remove samples with -1 or 0 optimal power

In [10]:
infile  = 'su_pu'
outfile = 'su_pu_remove'

with open(infile, 'r') as inpt, open(outfile, 'w') as output:
    for line in inpt:
        line = line.split(', ')
        opt = line[-1].strip()
        if opt != '-1' and opt != '0':
            output.write(', '.join(line))

infile  = 'su_ss'
outfile = 'su_ss_remove'

with open(infile, 'r') as inpt, open(outfile, 'w') as output:
    for line in inpt:
        line = line.split(', ')
        opt = line[-1].strip()
        if opt != '-1' and opt != '0':
            output.write(', '.join(line))

# 2 Calibrate Gains

In [17]:
def get_tx(x, y):
    if 0 <= x <= 4 and 0 <= y <= 4:
        return '3'
    if 5 <= x <= 9 and 0 <= y <= 4:
        return '2'
    if 5 <= x <= 9 and 5 <= y <= 9:
        return '1'
    if 0 <= x <= 4 and 5 <= y <= 9:
        return '4'
    raise Exception(f'{x} {y} not valid')

In [33]:
def calibrate(x, tx):
    '''Calibrated to TX3 (PU3)
    '''
    if tx == '1':
        return x - 50
    if tx == '2':
        return 1.3*x - 57.5
    if tx == '3':
        return x
    if tx == '4':
        return x - 27
    if tx == 'hackrf':
        return 1.03*x + 21.26
    raise Exception(f'tx = {tx}, not valid input')

In [41]:
infile  = 'su_pu_remove'
outfile = 'su_pu_calibrate'

with open(infile, 'r') as inpt, open(outfile, 'w') as output:
    for line in inpt:
        line = line.split(', ')
        num_pu = int(line[0])
        for i in range(num_pu):
            x = float(line[1 + i*3])
            y = float(line[1 + i*3 + 1])
            gain = float(line[1 + i*3 + 2])
            tx = get_tx(x, y)
            gain2 = calibrate(gain, tx)
            line[1 + i*3 + 2] = '{:.1f}'.format(gain2)
        opt = float(line[-1].strip())
        opt2 = calibrate(opt, 'hackrf')
        line[-1] =  '{:.1f}\n'.format(opt2)
        output.write(', '.join(line))


infile  = 'su_ss_remove'
outfile = 'su_ss_calibrate'

with open(infile, 'r') as inpt, open(outfile, 'w') as output:
    for line in inpt:
        line = line.split(', ')
        opt  = float(line[-1].strip())
        opt2 = calibrate(opt, 'hackrf')
        line[-1] =  '{:.1f}\n'.format(opt2)
        line.pop(4)                             # sensor 101 is down
        output.write(', '.join(line))

# 158, 160, 170, 180 all ways report noise floor
# So, 18 - 1 - 4 = 13 useful sensors